In [1]:
import pandas as pd

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df

,Lugar de residencia,Medida,Valor
0,Huelva,Población,510743
1,Huelva,Confirmados PDIA,9350
2,Huelva,Confirmados PDIA 14 días,2017
3,Huelva,Tasa PDIA 14 días,"394,91485933238437"
4,Huelva,Confirmados PDIA 7 días,827
...,...,...,...
627,Municipio de Huelva sin especificar,Tasa PDIA 14 días,-
628,Municipio de Huelva sin especificar,Confirmados PDIA 7 días,15
629,Municipio de Huelva sin especificar,Total Confirmados,122
630,Municipio de Huelva sin especificar,Curados,57


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  9350.0


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  2619.0


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 572 personas en los últimos 7 días 

Un positivo PCR cada 229 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,9350.0,827.0,2017.0,510743.0,161.920966,394.914859,178.0
Huelva-Costa,5121.0,562.0,1274.0,288115.0,195.061000,442.184544,125.0
Condado-Campiña,2892.0,210.0,577.0,155057.0,135.434066,372.121220,54.0
Huelva (capital),2619.0,251.0,627.0,143663.0,174.714436,436.438053,45.0
Lepe,822.0,119.0,227.0,27431.0,433.815756,827.530896,27.0
Aljaraque,351.0,37.0,77.0,21260.0,174.035748,362.182502,15.0
Palma del Condado (La),422.0,32.0,88.0,10761.0,297.370133,817.767865,12.0
Cartaya,290.0,32.0,78.0,19974.0,160.208271,390.507660,12.0
Isla Cristina,271.0,32.0,57.0,21264.0,150.489090,268.058691,12.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() > 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Corteconcepción,17.0,2.0,13.0,548.0,364.963504,2372.262774,0.0
Lepe,822.0,119.0,227.0,27431.0,433.815756,827.530896,27.0
Niebla,108.0,20.0,34.0,4117.0,485.790624,825.844061,10.0
Palma del Condado (La),422.0,32.0,88.0,10761.0,297.370133,817.767865,12.0
Lucena del Puerto,64.0,14.0,26.0,3371.0,415.307031,771.284485,4.0
Minas de Riotinto,62.0,4.0,25.0,3848.0,103.950104,649.688150,0.0
Rociana del Condado,103.0,19.0,51.0,7855.0,241.884150,649.267982,4.0
Ayamonte,295.0,54.0,122.0,20946.0,257.805786,582.450110,9.0
Paymogo,12.0,5.0,6.0,1159.0,431.406385,517.687662,0.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Puebla de Guzmán,21.0,1.0,10.0,3073.0,32.541490,325.414904,0.0,0.100000
Villablanca,54.0,1.0,10.0,2848.0,35.112360,351.123596,0.0,0.100000
Almonte,424.0,8.0,76.0,24191.0,33.070150,314.166426,0.0,0.105263
Aracena,280.0,2.0,13.0,8107.0,24.670038,160.355249,0.0,0.153846
Corteconcepción,17.0,2.0,13.0,548.0,364.963504,2372.262774,0.0,0.153846
Minas de Riotinto,62.0,4.0,25.0,3848.0,103.950104,649.688150,0.0,0.160000
Beas,51.0,3.0,18.0,4257.0,70.472163,422.832981,1.0,0.166667
Valverde del Camino,259.0,4.0,23.0,12820.0,31.201248,179.407176,1.0,0.173913
Trigueros,127.0,3.0,15.0,7713.0,38.895371,194.476857,3.0,0.200000
